<h1><font color="#15E5B9 " size=4>Text mining</font></h1>



<h1><font color="#15E5B9 " size=6>Generate dataset to predict 
 viewers' loyalty to a Twitch streamer</font></h1>



<div style="text-align: right">
<font size=3 color="GREY">Author: Juan Carlos Rivera</font><br>


---


</div>

In [ ]:
!pip install functools
from functools import reduce
import os
import json
import pandas as pd

**1. Read folder with Twitch strams chat's files and generate dataframes**

In [ ]:
directory = '/content/drive/My Drive/'
i=1
chats_log = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        data = json.load(open(f))
        print('Working on file', i)
        df = pd.json_normalize(data["comments"])
        users = df['commenter.display_name'].value_counts()
        users= users.index.tolist()
        globals()[f"chat{i}"] = pd.DataFrame (users, columns = ['user_name'])
        a = 'chat' + str(i)
        chats_log.append(globals()[f"chat{i}"])
        globals()[f"chat{i}"][a] = 1
        i=i+1

Working on file 1
Working on file 2
Working on file 3


**2. Merge all dataframes**

In [ ]:
merged_df = reduce(lambda  left,right: pd.merge(left,right,on=['user_name'], how='outer'), chats_log)

In [ ]:
merged_df = merged_df.fillna(0)

**3. Process data and generate dataframe with past fidelity and future fidelity (target)**

In [ ]:
u_names = []
prev_fid = []
fut_fid = []
for i in range (len(merged_df)):
  #print('Row', i)
  uname = merged_df.iloc[:, 0][i]
  #print('Name: ', uname)
  for j in range (len(merged_df.columns)):
    if (j != 0):
      field = merged_df.iloc[:, j][i]
      if (field == 1):
        split_uname = uname + '_' + merged_df.columns[j]
        u_names.append (split_uname)
        prev_fidelity = merged_df.iloc[:,1:j+1].mean(axis=1)
        prev_fidelity = prev_fidelity[i]
        fut_fidelity = merged_df.iloc[:,j+1:].mean(axis=1)
        fut_fidelity = fut_fidelity[i]
        prev_fid.append (prev_fidelity)
        fut_fid.append (fut_fidelity)

In [ ]:
output = zip(u_names,prev_fid, fut_fid)
output = pd.DataFrame(output, columns=['user_name','Previous fidelity', 'Target'])

In [ ]:
output